In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

# 记录一些PyTorch的常用API

### Embedding

In [3]:
'''Embed的最多数量(词典的数量)，向量的维度'''
vocab_size = 10
embed_dim = 10
# embedding = nn.Embedding(num_embeddings = vocab_size, embedding_dim = embed_dim)
embedding = nn.Embedding(vocab_size, embed_dim)
input1 = Variable(torch.LongTensor([1, 2, 3, 5]))
input2 = Variable(torch.LongTensor([1, 2, 3, 5, 7]))
input3 = Variable(torch.LongTensor([[1, 2, 3], [2, 3, 4]]))



print ("Embed1=", embedding(input1).size())
print ("Embed2=", embedding(input2).size())
print ("Embed3=", embedding(input3).size())

x = torch.LongTensor([2, 3]).unsqueeze(0)
input4 = torch.cat((x, x), 0)
#print (input4.size(), input4)
#print ("Embed4", embedding(input4))

('Embed1=', torch.Size([4, 10]))
('Embed2=', torch.Size([5, 10]))
('Embed3=', torch.Size([2, 3, 10]))


### GRU

In [4]:
'''
1. 初始化参数
(input_size, hidden_size, n_layers) 或者 不选最后一个

2. 输入
(input, hidden)
input: (seq_len, batch, input_size), 一般(1, 1, input_size)
hidden: (n_layers*n_dirs, batch, hidden_size), 一般(1, 1, hidden_size)

3. 输出
(output, hidden)
output: 最顶层的RNN输出 (seq_len, batch, hidden_size*n_dirs)，一般(1, b, hidden_size)
hidden: 时刻t=seq_len时的隐状态 (n_layers*n_dirs, batch, hidden_size)
'''

def test1():
    rnn = nn.GRU(10, 20, 2)
    input = Variable(torch.randn(5, 3, 10))
    h0 = Variable(torch.randn(2, 3, 20))
    output, h0 = rnn(input, h0)
    print ('osize=', output.size(), 'hsize=', h0.size())
test1()

# 也可以自己实现多层，加个循环就可以

('osize=', torch.Size([5, 3, 20]), 'hsize=', torch.Size([2, 3, 20]))


### unsqeeze

In [3]:
'''
1. torch.unsqueeze 解缩，增加维数。0在前面加维数，1在后面加维数
'''
#[4]
x = torch.Tensor([1, 2, 3, 4])
print (x.size())

# [1, 4]
y1 = torch.unsqueeze(x, 0)
print (y1.size())

# [4, 1]
y2 = torch.unsqueeze(x, 1)
print (y2.size())

# [1, 1, 4]
print (y1.unsqueeze(0).size())
# [1, 4, 1]
print (y2.unsqueeze(0).size())

torch.Size([4])
torch.Size([1, 4])
torch.Size([4, 1])
torch.Size([1, 1, 4])
torch.Size([1, 4, 1])


### squeeze压缩

In [7]:
'''
移除size=1的维数。0：移除第1维（size=1时移除）。不带参数：移除所有size=1的维数
'''
def test_squeeze():
    x = torch.zeros(2, 1, 3, 4, 5, 1)
    # 移除第0维的数据
    print (x.squeeze(0).size())
    # 默认移除所有size=1的维数
    #print (x.squeeze().size())
    # 不能移除其它维
    print (x.squeeze(1).size())
    # 只移除第1维size=1的
    x = torch.zeros(1, 1, 2)
    #print (x.squeeze(1).size())
test_squeeze()

torch.Size([2, 1, 3, 4, 5, 1])
torch.Size([2, 3, 4, 5, 1])


### transpose

In [26]:
'''
Tensor中的两个维度做交换
'''
def test_transpose():
    x = torch.randn(2, 3, 4)
    print (x.transpose(0, 1).size())
    print (x.transpose(0, 2).size())

test_transpose()

torch.Size([3, 2, 4])
torch.Size([4, 3, 2])


### dot

In [10]:
'''dot 需要是1维的tensor'''
x = torch.Tensor([2, 3])
y = torch.Tensor([2, 1])
print (x.dot(y))
x = torch.Tensor([[2, 3]])
y = torch.Tensor([[2, 3]])
print (x.size(), y.size(), y.t().size())
# 减到1维可以dot
print (x.squeeze(0).dot(y.squeeze()))

7.0
(torch.Size([1, 2]), torch.Size([1, 2]), torch.Size([2, 1]))
13.0


### bmm 矩阵相乘

In [5]:
'''4. torch.bmm 矩阵相乘的Tensor
bach1 b*n*m, batch2: b*m*p, 结果：b*n*p
'''
batch1 = torch.randn(10, 3, 4)
batch2 = torch.randn(10, 4, 5)
res = torch.bmm(batch1, batch2)
print (res.size())

torch.Size([10, 3, 5])


### topk

In [10]:
'''topk和max'''
x = torch.Tensor([[1,2],[3,4]]) # 5个数 [5]
print (x.size())
# topk 返回的是(values, indics)
values, indics = x[0].topk(1)
print (values)
print (type(indics), type(indics[0]))
# x.max() 会返回最大值，但是不会返回索引
print (Variable(x).size())

torch.Size([2, 2])

 2
[torch.FloatTensor of size 1]

(<class 'torch.LongTensor'>, <type 'int'>)
torch.Size([2, 2])


## pack_padded_sequence

In [ ]:
'''
input: T*B*?, T=maxlen, B=batch_size
要降序排列, input[:, 0]是最长的, input[:,B-1]是最短的
lengths: list[int]

output: PackedSequence ps.data可以访问到数据
可以用来pack labels, 或者使用RNN的output和它们一起直接计算loss
'''

torch.nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=False)

embedded = self.embedding(input_seqs)
# 塞满
packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
outputs, hidden = self.gru(packed, hidden)
# 拉长
outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs) # unpack (back to padded)

### 访问3维元素

In [ ]:
def test_slice():
    a = torch.randn(2, 3, 4)
    print (a)

### BMM矩阵相乘

In [4]:
'''4. torch.bmm 矩阵相乘的Tensor
bach1 b*n*m, batch2: b*m*p, 结果：b*n*p
'''
batch1 = torch.randn(10, 3, 4)
batch2 = torch.randn(10, 4, 5)
res = torch.bmm(batch1, batch2)
print (res.size())

torch.Size([10, 3, 5])


### cat

In [9]:
'''3. torch.cat 把多个tensor组合在一起。形状要统一一样
'''
x = torch.randn(2, 3)
# 按行排列 6*3
print (torch.cat((x, x, x), 0).size())
# 按列排 2*9
print (torch.cat((x, x, x), 1).size())
y = torch.randn(3, 3)

torch.Size([6, 3])
torch.Size([2, 9])


In [21]:
import visdom
import numpy as np
vis = visdom.Visdom()
vis.text('Hello, world!')
vis.image(np.ones((3, 10, 10)))

u'pane_35946a89a899aa'